In [1]:
## Import python library
import pandas as pd
import nltk

## Read file
file_name = "data/train.csv"
## Read file using pandas
df = pd.read_csv(file_name)

In [3]:
!pip install textblob

In [4]:
## Function for augmenting data using langauge translation
## Could not found free service for langauge translation, Use paid service like Azure, Google translator etc

from textblob import TextBlob
from textblob.translate import NotTranslated
import random
sr = random.SystemRandom()

language = ["es", "de", "fr", "ar", "te", "hi", "ja", "fa", "sq", "bg", "nl", "gu", "ig", "kk", "mt", "ps"]

def data_augmentation(message, language, aug_range=1):
    augmented_messages = []
    if hasattr(message, "decode"):
        message = message.decode("utf-8")

    for j in range(0,aug_range) :
        new_message = ""
        text = TextBlob(message)
        try:
            text = text.translate(to=sr.choice(language))   ## Converting to random langauge for meaningful variation
            text = text.translate(to="en")
        except NotTranslated:
            pass
        augmented_messages.append(str(text))

    return augmented_messages

In [5]:
## Dictionary for intent count
## Intent is column name
intent_count = df.jobflag.value_counts().to_dict()

In [6]:
## Get max intent count to match other minority classes through data augmentation
import operator
max_intent_count = max(intent_count.items(), key=operator.itemgetter(1))[1]

In [11]:
## Loop to interate all messages
import numpy as np
import math
import tqdm
newdf = pd.DataFrame()
for intent, count in intent_count.items() :
    count_diff = max_intent_count - count    ## Difference to fill
    multiplication_count = math.ceil((count_diff)/count)  ## Multiplying a minority classes for multiplication_count times
    if (multiplication_count) :
        old_message_df = pd.DataFrame()
        new_message_df = pd.DataFrame()
        for message in tqdm.tqdm(df[df["jobflag"] == intent]["description"]) :
            ## Extracting existing minority class batch
            dummy1 = pd.DataFrame([message], columns=['description'])
            dummy1["jobflag"] = intent
            old_message_df = old_message_df.append(dummy1)
            
            ## Creating new augmented batch from existing minority class
            new_messages = data_augmentation(message, language, multiplication_count)
            dummy2 = pd.DataFrame(new_messages, columns=['description'])
            dummy2["jobflag"] = intent
            new_message_df = new_message_df.append(dummy2)
        
        ## Select random data points from augmented data
        new_message_df=new_message_df.take(np.random.permutation(len(new_message_df))[:count_diff])
        
        ## Merge existing and augmented data points
        newdf = newdf.append([old_message_df,new_message_df])
    else :
        newdf = newdf.append(df[df["jobflag"] == intent])


  6%|██████████▎                                                                                                                                                                            | 35/624 [02:14<36:49,  3.75s/it]


 11%|████████████████████▊                                                                                                                                                                  | 71/624 [04:29<34:51,  3.78s/it]


 17%|███████████████████████████████▏                                                                                                                                                      | 107/624 [06:50<33:28,  3.89s/it]


 23%|█████████████████████████████████████████▋                                                                                                                                            | 143/624 [09:09<30:11,  3.77s/it]


 29%|████████████████████████████████████████████████████▏                                                                                                                                 | 179/624 [11:24<24:48,  3.34s/it]


...

 33%|██████████████████████▊                                              | 115/348 [10:04<21:27,  5.53s/it]


 54%|█████████████████████████████████████▎                               | 188/348 [16:25<13:01,  4.89s/it]


 75%|███████████████████████████████████████████████████▊                 | 261/348 [22:57<07:58,  5.50s/it]


 96%|██████████████████████████████████████████████████████████████████▏  | 334/348 [29:30<01:13,  5.26s/it]


100%|█████████████████████████████████████████████████████████████████████| 348/348 [30:44<00:00,  5.30s/it]


In [13]:
## Print count of all new data points
newdf.jobflag.value_counts()

3    1376
2    1376
1    1376
4    1376
Name: jobflag, dtype: int64

In [15]:
## Save newdf back to file
newdf.to_csv("data/data_augmentation_using_language_translation.csv", index=False)